#### Crear la Tabla results_movie en la capa "gold"

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
spark.sql("""
          CREATE TABLE IF NOT EXISTS movie_gold.results_movie(
              year_release_date INT,
              country_name STRING,
              company_name STRING,
              budget FLOAT,
              revenue FLOAT,
              movie_id INT,
              country_id INT,
              company_id INT,
              created_date DATE,
              updated_date DATE
          )
          USING DELTA
          """)

In [0]:
%python
spark.sql(f"""
        CREATE OR REPLACE TEMP VIEW v_results_movie
        AS
        SELECT year_release_date, country_name, company_name , budget, revenue,M.movie_id,C.country_id,PCO.company_id
        FROM movie_silver.MOVIES AS M
        INNER JOIN movie_silver.PRODUCTIONS_COUNTRIES AS PC ON (M.MOVIE_ID = PC.MOVIE_ID)
        INNER JOIN movie_silver.COUNTRIES AS C ON (PC.COUNTRY_ID = C.COUNTRY_ID)
        INNER JOIN movie_silver.MOVIES_COMPANIES AS MC ON (M.MOVIE_ID = MC.MOVIE_ID)
        INNER JOIN movie_silver.PRODUCTIONS_COMPANIES AS PCO ON (MC.COMPANY_ID = PCO.COMPANY_ID)
        WHERE M.file_date =  '{v_file_date}'
""")

In [0]:
%sql
SELECT *
FROM v_results_movie;

In [0]:
spark.sql(f"""
        MERGE INTO movie_gold.results_movie AS tgt
        USING v_results_movie AS src
        ON (tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.company_id = src.company_id)
        WHEN MATCHED THEN
        UPDATE SET
            tgt.year_release_date = src.year_release_date,
            tgt.country_name = src.country_name,
            tgt.company_name = src.company_name,
            tgt.budget = src.budget,
            tgt.revenue = src.revenue,
            tgt.updated_date = current_timestamp
        WHEN NOT MATCHED
        THEN INSERT (year_release_date, country_name, company_name, budget, revenue, movie_id, country_id, company_id, created_date)
            VALUES (year_release_date, country_name, company_name, budget, revenue, movie_id, country_id, company_id, current_timestamp)
""")

In [0]:
%sql
SELECT count(1)
FROM v_results_movie;

In [0]:
%sql
SELECT count(1)
FROM MOVIE_GOLD.results_movie;